In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import itertools

In [ ]:
def metric(df, preds):
    df["prediction"] = preds
    maes = []
    for t in df.type.unique():
        y_true = df[df.type==t].scalar_coupling_constant.values
        y_pred = df[df.type==t].prediction.values
        mae = np.log(metrics.mean_absolute_error(y_true, y_pred))
        maes.append(mae)
    return np.mean(maes)

## Read data

In [ ]:
# Groups all dataframes according to the molecule and returns
# result as a dictionary
def group_structures(df, struct_df, mulliken_df, potential):
    groups = {}
    struct_g = struct_df.groupby('molecule_name')
    mulliken_g = mulliken_df.groupby('molecule_name')
    for g, gdf in df.groupby('molecule_name'):
        groups[g] = (gdf, struct_g.get_group(g), 
                     mulliken_g.get_group(g), 
                     potential[potential.molecule_name == g])
    return groups

In [ ]:
import os.path as osp
req_files = ['train.csv', 'structures.csv', 'mulliken_charges.csv', 'potential_energy.csv']
def read_csvs(path):
    read_csv = lambda x: pd.read_csv(osp.join(path, x))
    return tuple(map(read_csv, req_files))

# Reads files from path and returns a zipped list of the data according
# to the molecule
def get_data_list(path):
    train_df, structures, mulliken, potential = read_csvs(path)
    train_df['type'] = train_df['type'].astype('category')
    train_df['type_c'] = train_df['type'].cat.codes
    structures['atom'] = structures['atom'].astype('category')
    structures['atom_c'] = structures['atom'].cat.codes
    return list(group_structures(train_df, structures, mulliken, potential).values())

In [ ]:
def to_data(first):
    # Edge Attributes
    # Edges are bidirectional
    src, dst = first[0].atom_index_0, first[0].atom_index_1
    src, dst = np.concatenate((src, dst)), np.concatenate((dst, src))
    edge_idx = np.stack((src, dst))
    
    scalar_coupling = first[0].scalar_coupling_constant.values
    # Edge types
    edge_types = np.concatenate((first[0].type_c.values, first[0].type_c.values))
    
    # Atom Attributes
    xyz, atom = first[1].iloc[:,3:-1].values, first[1].iloc[:,-1].values
    mul_charge = first[2].iloc[:,-1].values
    
    data = Data(pos=torch.FloatTensor(xyz), 
                edge_index=torch.LongTensor(edge_idx), 
                edge_types=torch.LongTensor(edge_types),
                atom=torch.LongTensor(atom),
                charge=torch.FloatTensor(mul_charge),
                energy=torch.FloatTensor(first[3].potential_energy.values),
                batch_edge_index=torch.zeros(edge_types.shape, dtype=torch.long),
                half_edge_index=torch.LongTensor(edge_idx[:, :edge_idx.shape[1] // 2]),
                scalar_coupling=torch.FloatTensor(scalar_coupling))
    return data

In [ ]:
import numpy as np

# Given two numpy arrays A and B, returns A - B, considering columns
# of A and B to be elements in a set
def set_diff(A, B):
    A, B = np.ascontiguousarray(A.T), np.ascontiguousarray(B.T)
    nrows, ncols = A.shape
    
    dtype={'names':['f{}'.format(i) for i in range(ncols)],
           'formats':ncols * [A.dtype]}
    C = np.setdiff1d(A.view(dtype), B.view(dtype))
    C = C.view(A.dtype).reshape(-1, ncols)
    
    #print(C.dtype)
    return np.ascontiguousarray(C.T)

In [ ]:
# Graph transformation to make a graph a complete graph
class Complete(object):
    def __init__(self):
        pass
    
    def __call__(self, data):
        complete_edges = np.array(list(itertools.permutations(range(data.num_nodes),2))).T
        # additional edges needed to make a complete graph
        new_edges = torch.LongTensor(set_diff(complete_edges, data.edge_index)).contiguous()
        
        # label all additional edges as type 8 (UNK)
        new_types = 8 * np.ones(new_edges.shape[1])
        
        data.prev_edge_index = data.edge_index
        data.edge_index = torch.cat((data.edge_index, new_edges), axis=1)
        
        data.full_types = torch.LongTensor(np.concatenate((data.edge_types, new_types)))#.cuda()
        return data

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, Data
import torch_geometric.transforms as T

class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyOwnDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return ['structures.csv', 'mulliken_charges.csv', 'train.csv', 'magnetic_shielding_tensors.csv']
    
    @property
    def processed_file_names(self):
        return ['data_tmp.pt']
    
    def _download(self):
        pass
    
    def process(self):
        data_list = get_data_list(self.root)
        data_list = [to_data(data) for data in data_list]
        
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]
        
        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
#!rm data/processed/data.pt

In [ ]:
#rm data/processed/data_tmp.pt

In [ ]:
dataset = MyOwnDataset('data', transform=T.Compose([Complete(), T.Distance()]))#, transform=Complete())

In [ ]:
normalize = False

In [ ]:
# Normalize targets to mean=0 and std=1
if normalize:
    mean = dataset.data.energy.mean(dim=0, keepdim=True)
    std = dataset.data.energy.std(dim=0, keepdim=True)
    dataset.data.energy = (dataset.data.energy - mean) / std

In [ ]:
# Normalize targets to mean=0 and std=1
if normalize:
    mean = dataset.data.scalar_coupling.mean(dim=0, keepdim=True)
    std = dataset.data.scalar_coupling.std(dim=0, keepdim=True)
    dataset.data.scalar_coupling = (dataset.data.scalar_coupling - mean) / std
    print(mean, std)

In [ ]:
plt.plot(dataset.data.scalar_coupling)

In [ ]:
train_mask = torch.FloatTensor(len(dataset)).uniform_() > 0.3

In [ ]:
train_mask.sum() / float(train_mask.size(0))

In [ ]:
train_dataset = dataset[train_mask]
valid_dataset = dataset[~train_mask]

## Simple Model

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import NNConv, Set2Set, GCNConv
from torch_geometric.data import DataLoader

from torch.nn import Sequential, Linear, ReLU, GRU, Embedding, LeakyReLU, BatchNorm1d

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=500, shuffle=True)

In [ ]:
dim = 64
POS_SIZE = 20
ATOM_SIZE = 20
EDGE_SIZE = 25
DIST_SIZE = 20

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        def seq_emb(in_, out_, fl):
            return Sequential(fl(in_, out_),
                              LeakyReLU(),
                              BatchNorm1d(out_))
        
        self.lin0 = torch.nn.Linear(POS_SIZE + ATOM_SIZE, dim)
        
        self.pos_emb = seq_emb(3, POS_SIZE, Linear)
        self.atom_emb = seq_emb(5, ATOM_SIZE, Embedding)
        self.edge_emb = seq_emb(9, EDGE_SIZE, Embedding) 
        self.dist_emb = seq_emb(1, DIST_SIZE, Linear)
        nn = Sequential(Linear(EDGE_SIZE + DIST_SIZE, 128), LeakyReLU(0.1), BatchNorm1d(128), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean')
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=6)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        pos = self.pos_emb(data.pos)
        atom_emb = self.atom_emb(data.atom)
        x = torch.cat((pos, atom_emb), dim=1)
        out = F.relu(self.lin0(x))
        h = out.unsqueeze(0)
        
        edge_emb = self.edge_emb(data.full_types)
        edge_dist = self.dist_emb(data.edge_attr)
        edge_attr = torch.cat((edge_emb, edge_dist), dim=1)

        for i in range(6):
            m = F.relu(self.conv(out, data.edge_index, edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
            
        x = torch.index_select(out, 0, data.half_edge_index.T.contiguous().view(-1))
        x = x.view((data.half_edge_index.shape[1], -1))

        #out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(x))
        out = self.lin2(out)
        return out.view(-1)

In [ ]:
import pytorch_lightning as pl

class LightningNet(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.net = Net()
        self.lr = hparams.learning_rate
        self.hparams = hparams
        
    def forward(self, data):
        return self.net(data)
    
    def training_step(self, batch, batch_idx):
        output = self.forward(batch)
        loss = torch.log(F.mse_loss(output, batch.scalar_coupling))
        return {'loss': loss, 'progress_bar': {'training_loss': loss}}
    
    def validation_step(self, batch, batch_idx):
        output = self.forward(batch)
        loss = torch.log(F.mse_loss(output, batch.scalar_coupling))
        return {'loss': loss}
    
    def validation_end(self, outputs):
        avg_loss = torch.mean(torch.Tensor([x['loss'] for x in outputs]))
        return {'val_loss': avg_loss,
                'progress_bar': {'valid_loss': avg_loss},
                'log': {'valid_loss': avg_loss}}
        
    def configure_optimizers(self):
        optim = torch.optim.AdamW(self.parameters())
        lr_sch = torch.optim.lr_scheduler.OneCycleLR(optim, self.lr, 
                                                     len(train_loader),
                                                     epochs=30)
        return [optim], [lr_sch]
    
    @pl.data_loader
    def train_dataloader(self):
        return train_loader
    
    @pl.data_loader
    def val_dataloader(self):
        return valid_loader

In [ ]:
from argparse import Namespace

# usually these come from command line args
args = Namespace(**{'learning_rate':0.001})
net = LightningNet(args)

In [ ]:
b = next(iter(train_loader))

In [ ]:
b.half_edge_index[:100]

In [ ]:
b.edge_index

In [ ]:
net.cpu()
net(b)

In [ ]:
early_stop_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)

trainer = pl.Trainer(gpus=1, 
                     early_stop_callback=early_stop_callback)
trainer.fit(net)

In [ ]:
net.lr = 1e-3

In [ ]:
net.lr

In [ ]:
net = LightningNet.load_from_checkpoint('../kaggle/champs/lightning_logs/version_77/checkpoints/_ckpt_epoch_12.ckpt')

In [ ]:
!ls -lt lightning_logs/**/**/*.ckpt

In [ ]:
from torch.utils.data import Dataset
class DS(Dataset):
    def __init__(self, ds):
        self.ds = ds
    
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        x = self.ds[idx]
        return x, x.scalar_coupling

In [ ]:
crit = lambda y_pred, y: torch.log(F.mse_loss(y_pred, y))
lrf = LRFinder(net, torch.optim.AdamW(net.parameters()), crit, device='cpu')

In [ ]:
lrf.range_test(DS(train_dataset), start_lr=1e-6, end_lr=0.001, num_iter=100)
lrf.plot() # to inspect the loss-learning rate graph

In [ ]:
net.hparams

In [ ]:
net.load_from_checkpoint('first.ckpt')